In [1]:
import pandas as pd
import numpy as np
import duckdb
import warnings
import os

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# email
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

import warnings
from datetime import datetime, timedelta
from haversine import haversine

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']
con = duckdb.connect(database=":memory:")

cwd = os.path.abspath(os.getcwd()).replace("\\", "/")
if cwd.startswith("C:/Users/Rodolfo/"):
    RUN_LOCATION = "local"
else:
    RUN_LOCATION = "cloud"
time_offset = {"local": 3, "cloud": -5}
now = str((datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]) + timedelta(hours=-3)).date())
print(f"Today's date:", now)

Today's date: 2026-01-05


In [2]:
%run ./common_utils.ipynb

# Initial Functions

In [3]:
def email(model, error):
    
    # Email details
    sender_email = "rodolfoe7157@gmail.com"
    receiver_email = "rodolfoe7157@gmail.com"
    password = "cqgu bfey cnyx sfue"  # See note below

    subject = "NBA create_Predictions error"
    body = f"Model: {model}_model\nERROR: {error}"

    # Create message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    # Connect to Gmail SMTP server and send
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(sender_email, password)
        server.send_message(msg)

    print("Email sent successfully!")

In [4]:
def create_base_df():
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    df3 = pd.DataFrame()
    df4 = pd.DataFrame()
    for i in [2022, 2023, 2024, 2025]:
        df_actuals = pd.read_csv(f"../tables/{i}/parlay_stats.csv")
        df_actuals['Season'] = i
        df = pd.concat([df, df_actuals])

        df_schd = pd.read_csv(f"../tables/{i}/nba_schedule.csv")
        df_schd['Season'] = i
        df2 = pd.concat([df2, df_schd])

        df_gms = pd.read_csv(f"../tables/{i}/season_gamelogs.csv")
        df_gms['Season'] = i
        df3 = pd.concat([df3, df_gms])

        df_inj = pd.read_csv(f"../tables/{i}/injuries.csv")
        df_inj['Season'] = i
        df4 = pd.concat([df4, df_inj])

    df['Date'] = pd.to_datetime(df.Date)
    df2['Date'] = pd.to_datetime(df2.Date)
    df3['Date'] = pd.to_datetime(df3.Date)
    df3 = df3[~df3[['Date', 'Team', 'Player']].duplicated(keep='last')]
    df4['Date'] = pd.to_datetime(df4.Date)

    df3_temp = df3.rename(columns={"3PM": "TPM", "3PA": "TPA", "3P%": "TP%", "TRB": "REB"}).drop(['Pos', 'Opp'], axis=1)
    df3_temp['PR'] = df3_temp.PTS + df3_temp.REB 
    df3_temp['PA'] = df3_temp.PTS + df3_temp.AST
    df3_temp['RA'] = df3_temp.REB + df3_temp.AST
    df3_temp['PRA'] = df3_temp.PTS + df3_temp.REB + df3_temp.AST
    df3_temp['STL_BLK'] = df3_temp.STL + df3_temp.BLK
    df = df.merge(df3_temp, on=['Season', 'Date', 'Team', 'Player'], how='left')

    df_mtch = df2[['Season', 'Date', 'AwayABV', 'HomeABV', 'AwayPTS', 'HomePTS', 'AwayB2B', 'HomeB2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch['Team_type'] = 'Away'
    df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm', 'Team_type']]
    df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch2['Team_type'] = 'Home'
    df_mtch = pd.concat([df_mtch, df_mtch2])
    df_mtch = df_mtch[['Season', 'Date', 'Team', 'Team_type', 'AwayPTS', 'HomePTS', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch = df_mtch.sort_values(["Team", "Date"])
    df_mtch['team_game_num'] = df_mtch.groupby(["Team", "Season"]).cumcount() + 1
    df_mtch['Spread'] = np.where(df_mtch.Team_type == 'Home', df_mtch.HomePTS - df_mtch.AwayPTS, df_mtch.AwayPTS - df_mtch.HomePTS)
    df_mtch['Total'] = df_mtch.AwayPTS + df_mtch.HomePTS
    df_mtch['is_Win'] = np.where(df_mtch.Spread > 0, 1, 0)
    df_mtch['Szn_Wins'] = df_mtch.groupby(['Season', 'Team'])['is_Win'].cumsum()
    df = df.drop(['Season', 'Team_type'], axis=1).merge(df_mtch, on=['Date', 'Team'])

    df = df.merge(df4[['Date', 'Team', 'Player', 'Status']], on=['Date', 'Team', 'Player'], how='left')
    df['Status'] = np.where((df.Active == 1) & (df.Status.isnull()), 'Available', df.Status)
    df['Status'] = np.where((df.Active == 0), 'Out', df.Status)
    df['Status'] = np.where((df.Status == 'Out') & (df.Active != 0), 'Available', df.Status)
    
    return df

# Minutes Projection Model

In [5]:
def setup_df_mins(con, df):
    
    df = df[['Season', 'Date', 'Team', 'Team_type', 'Opp', 'Player', 'Pos', 'B2B', 'MP',
             'Spread', 'Total', 'team_game_num', 'Szn_Wins', 'pstszn_gm', 'is_OT']]
    
    for col in ['MP']:
        for N in [1, 3, 5, 10]:
            df[f'{col}_L{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )

    games_last_7_days = df.sort_values(['Player', 'Season', 'Date']).groupby(['Player', 'Season']).rolling('7D', on='Date', closed='left')['MP'].count().reset_index().rename(columns={"MP": "gms_L7_days"})
    games_last_7_days = games_last_7_days.drop_duplicates(
        subset=['Player', 'Season', 'Date']
    )
    df = df.merge(games_last_7_days, on=['Player', 'Season', 'Date'])
    df['gms_L7_days'] = df.gms_L7_days.fillna(0).astype(int)
    
    df['prev_team_mins_pct'] = (df.groupby(['Player', 'Season'])['MP'].shift(1)) / 240
           
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
    
    for N in [1, 3, 5]:
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df[f'{role}_last{N}'] = (
                df.sort_values(['Player', 'Date']).groupby('Player')[role]
                  .rolling(N, min_periods=1)
                  .sum()
                  .shift(1)
                  .reset_index(0, drop=True)
            )
        rec_role_cols = [f'{role}_last{N}' for role in ['reserve_td', 'bench_td', 'starter_td']]
        df[f'recent_most_common_role_L{N}'] = df[rec_role_cols].idxmax(axis=1)
        df[f'recent_role_L{N}'] = 0
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'starter_td_last{N}', 1, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'bench_td_last{N}', 2, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'reserve_td_last{N}', 3, df[f'recent_role_L{N}'])      
        df = df.drop(f'recent_most_common_role_L{N}', axis=1)
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df = df.drop(f'{role}_last{N}', axis=1)
      
    df['missed_games'] = (
        df.groupby(['Player', 'Team', 'Season'])['team_game_num']      
          .diff()
          .sub(1)
          .fillna(0)
          .astype(int)
    )
    
    df['game_spread_type'] = 0
    df['game_spread_type'] = np.where(abs(df.Spread < 6), 1, df.game_spread_type) 
    df['game_spread_type'] = np.where((abs(df.Spread >= 6) & abs(df.Spread <= 14)), 2, df.game_spread_type) 
    df['game_spread_type'] = np.where(abs(df.Spread > 14), 3, df.game_spread_type) 
    
    df['missed_games_after'] = (
        df.groupby(['Player', 'Team', 'Season'])['team_game_num']
          .diff(-1)
          .abs()
          .sub(1)
          .fillna(0)
          .astype(int)
    )
    df['MP_diff_next_avg5'] = df.groupby('Player')['MP'].shift(-1) - df['MP_L5_avg']
    df['Injured'] = (
                        ((df.missed_games_after > 0) & (df.MP_diff_next_avg5 < -10) & (df.role == 1)) | 
                        ((df.missed_games_after > 0) & (df.MP_diff_next_avg5 < -5) & (df.role == 2))
                    ).astype(int)
    
    df['return_game'] = (df.groupby('Player')['Injured'].shift(1) == 1).astype(int)
    df['games_since_return'] = (df.groupby('Player')['return_game'].cumsum())
    df['games_since_return'] = (df.groupby(['Player', 'games_since_return']).cumcount())
    df['ramp_phase'] = 0
    df.loc[df.return_game == 1, 'ramp_phase'] = 1
    df.loc[df.games_since_return.isin([1, 2]), 'ramp_phase'] = 2
    df.loc[df.games_since_return >= 3, 'ramp_phase'] = 3
    df['starter_return'] = ((df.return_game == 1) & (df.role == 1)).astype(int)
    df['bench_return']   = ((df.return_game == 1) & (df.role == 2)).astype(int)
    
    df['MP_drop_pct_L5'] = (df.MP_L5_avg - df.MP) / df.MP_L5_avg
    df['Early_stop'] = (
        (df['MP'] < 5) |
        ((df.role == 1) & (df.MP_drop_pct_L5 >= 0.35)) |   # starters
        ((df.role == 2) & (df.MP_drop_pct_L5 >= 0.45)) |   # bench
        ((df.role == 3) & (df.MP_drop_pct_L5 >= 0.55))     # reserve
    ).astype(int)
    
    for N in [1, 3, 5]:
        df[f'Early_stop_L{N}'] = df.sort_values(['Player', 'Date'])['Early_stop'].shift(1).rolling(N).sum()
        df[f'Injured_L{N}'] = df.sort_values(['Player', 'Date'])['Injured'].shift(1).rolling(N).sum()
    
    # Location based features
    df["DaysLstGm"] = (df.groupby("Player")["Date"].diff().dt.days).fillna(0).astype(int)
    df['Location'] = df.apply(lambda r: r['Team'] if r['Team_type'] == 'Home' else r['Opp'], axis=1)
    df['PrevLocation'] = df.groupby('Player')['Location'].shift(1)
    df['same_arena'] = (df['PrevLocation'] == df['Location']).astype(int)

    df = df.drop(['Season', 'Team_type', 'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter', 'MP_drop_pct_L5',
                  'PrevLocation', 'Location', 'Injured', 'missed_games_after', 'MP_diff_next_avg5', 'return_game'], axis=1)    
    
    return df

# Main Model

In [6]:
def setup_df_main(df, tgt_stat):
    
    # Stat dependent features 
    if tgt_stat == 'PTS':
        tgt_stat_cols = ['TPM', 'FG', 'FT', 'TPA', 'FGA', 'FTA']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 'team_game_num', 
         'PTS', 'TPM', 'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        # Efficiency metrics
        df['three_rate_raw'] =  np.where(df.FGA > 0, df['TPA'] / df['FGA'], 0)
        df['ft_rate_raw']    =  np.where(df.FGA > 0, df['FTA'] / df['FGA'], 0)
        df['eFG_raw'] = (df['FG'] + 0.5 * df['TPM']) / df['FGA']
        df['TS_raw'] = df['PTS'] / (2 * (df['FGA'] + 0.44 * df['FTA']))    
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        
        for w in [3, 5, 10]:
            for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
                col = f"{metric}_L{w}"
                df[col] = (
                    df.groupby(['Player','Season'])[f'{metric}_raw']
                      .rolling(w, min_periods=1)
                      .mean()
                      .shift(1)
                      .reset_index(level=[0,1], drop=True)
                )
        for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
            col = f'{metric}_weighted'
            df[col] = (
                0.6 * df[f'{metric}_L3'] +
                0.3 * df[f'{metric}_L5'] +
                0.1 * df[f'{metric}_L10']
            )
            df = df.drop(f'{metric}_raw', axis=1)
        
    elif tgt_stat == 'PRA':
        tgt_stat_cols = ['PTS', 'REB', 'AST', 'TPM', 'FG']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 'team_game_num', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK', 
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        for w in [3, 5, 10]:
            df[f"usage_proxy_L{w}"] = (
                df.groupby(['Player','Season'])[f'usage_proxy_raw']
                  .rolling(w, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0,1], drop=True)
            )
        df['usage_proxy_weighted'] = (
            0.6 * df[f'usage_proxy_L3'] +
            0.3 * df[f'usage_proxy_L5'] +
            0.1 * df[f'usage_proxy_L10']
        )
        df = df.drop('usage_proxy_raw', axis=1)
        
        
    else:
        tgt_stat_cols = []
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 'team_game_num', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
          f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
          f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]

    
    # Create rolling + lag features    
    for col in ['MP'] + tgt_stat_cols:
        df[f'{col}_lst_gm'] = (
            df
            .groupby(['Player', 'Season'])[col]
            .shift(1)
        )
        for N in [1, 3, 5]:
            df[f'{col}_L{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )

    # Role identifiers features
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
    
    for N in [1, 3, 5]:
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df[f'{role}_last{N}'] = (
                df.sort_values(['Player', 'Date']).groupby('Player')[role]
                  .rolling(N, min_periods=1)
                  .sum()
                  .shift(1)
                  .reset_index(0, drop=True)
            )
        rec_role_cols = [f'{role}_last{N}' for role in ['reserve_td', 'bench_td', 'starter_td']]
        df[f'recent_most_common_role_L{N}'] = df[rec_role_cols].idxmax(axis=1)
        df[f'recent_role_L{N}'] = 0
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'starter_td_last{N}', 1, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'bench_td_last{N}', 2, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'reserve_td_last{N}', 3, df[f'recent_role_L{N}'])      
        df = df.drop(f'recent_most_common_role_L{N}', axis=1)
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df = df.drop(f'{role}_last{N}', axis=1)
    
    df['game_spread_type'] = 0
    df['game_spread_type'] = np.where(abs(df.Spread < 6), 1, df.game_spread_type) 
    df['game_spread_type'] = np.where((abs(df.Spread >= 6) & abs(df.Spread <= 14)), 2, df.game_spread_type) 
    df['game_spread_type'] = np.where(abs(df.Spread > 14), 3, df.game_spread_type) 

    df['MP_drop_pct_L5'] = (df.MP_L5_avg - df.MP) / df.MP_L5_avg
    df['Early_stop'] = (
        (df['MP'] < 5) |
        ((df.role == 1) & (df.MP_drop_pct_L5 >= 0.35)) |   # starters
        ((df.role == 2) & (df.MP_drop_pct_L5 >= 0.45)) |   # bench
        ((df.role == 3) & (df.MP_drop_pct_L5 >= 0.55))     # reserve
    ).astype(int)
    
    
    for col in categories + ['Season', 'FG', 'FGA', 'FT', 'FTA', 'TPM', 'TPA', 'MP_drop_pct_L5', 
                             'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter'] + tgt_stat_cols:
        if col == tgt_stat:
            continue
        if col in df.columns:
            df = df.drop(col, axis=1)
        
    return df

### Today's predictions

In [10]:
def generate_predictions(tgt_stat):
    team_encoder = LabelEncoder()
    player_encoder = LabelEncoder()
    team_type_encoder = LabelEncoder()
    position_encoder = LabelEncoder()
    status_encoder = LabelEncoder()
    
    df_pred = create_base_df()
    
    # Encode string cols
    team_encoder.fit(pd.concat([df_pred["Team"], df_pred["Opp"]], axis=0))
    df_pred["Team"] = team_encoder.transform(df_pred["Team"])
    df_pred["Opp"] = team_encoder.transform(df_pred["Opp"])
    df_pred["Player"] = player_encoder.fit_transform(df_pred["Player"])
    df_pred["Pos"] = position_encoder.fit_transform(df_pred["Pos"])
    df_pred['Team_type'] = team_type_encoder.fit_transform(df_pred['Team_type'])
    df_pred["Status"] = status_encoder.fit_transform(df_pred["Status"])
    
    mins_model = xgb.XGBRegressor()
    mins_model.load_model("../ML_models/mins_model.json")
    stat_model = xgb.XGBRegressor()
    stat_model.load_model(f"../ML_models/{tgt_stat}_model.json")
    
    df_lines = pd.read_csv(f"../tables/2025/parlay_lines.csv")
    df_lines['Date'] = pd.to_datetime(df_lines.Date)
    df_lines = df_lines[~(df_lines.Team.isnull())]

    # Predict Mins
    df_lines["Team"] = team_encoder.transform(df_lines["Team"])
    df_pred = df_pred.merge(df_lines[['Date', 'Team', 'Spread', 'Total']], on=['Date', 'Team'], how='left')
    df_pred = df_pred[~df_pred[['Date', 'Team', 'Player']].duplicated(keep='last')]
    df_pred['Spread_x'] = np.where(df_pred.Spread_x.isnull(), df_pred.Spread_y, df_pred.Spread_x)
    df_pred['Total_x'] = np.where(df_pred.Total_x.isnull(), df_pred.Total_y, df_pred.Total_x)
    df_pred = df_pred.rename(columns={"Spread_x": "Spread", "Total_x": "Total"}).drop(['Spread_y', 'Total_y'], axis=1)
    df_pred_mins = setup_df_mins(con, df_pred)
    df_pred_mins = df_pred_mins.drop(['Date', 'MP'], axis=1)
    df_pred['MP'] = mins_model.predict(df_pred_mins)

    # Predict Stat
    df_pred = setup_df_main(df_pred, tgt_stat)
    feature_cols = [col for col in df_pred.columns if col not in ['Date', tgt_stat]]
    df_pred = df_pred[df_pred.Date == now][feature_cols]
    df_pred[f"{tgt_stat}_proj"] = stat_model.predict(df_pred)

    df_pred['Team'] = team_encoder.inverse_transform(df_pred["Team"])
    df_lines['Team'] = team_encoder.inverse_transform(df_lines["Team"])
    df_pred['Opp'] = team_encoder.inverse_transform(df_pred["Opp"])
    df_pred['Player'] = player_encoder.inverse_transform(df_pred["Player"])
    df_pred['Pos'] = position_encoder.inverse_transform(df_pred["Pos"])

    df_lines = df_lines[df_lines.Date == now][['Team', 'Player', f'{tgt_stat}_line']]
    df_pred = df_pred.merge(df_lines, on=['Team', 'Player'])

    tds_picks = df_pred[~(df_pred[f'{tgt_stat}_line'].isnull())]\
                [['Team', 'Player', 'Pos', 'Opp', 'MP', 'MP_L5_avg', f'{tgt_stat}_line', f'{tgt_stat}_proj']]
    tds_picks['Diff'] = abs((df_pred[f'{tgt_stat}_line'] - df_pred[f'{tgt_stat}_proj']))
    tds_picks['Diff2'] = abs((df_pred['MP'] - df_pred['MP_L5_avg']))
    tds_picks = tds_picks.sort_values('Diff', ascending=False).drop(['Diff', 'Diff2'], axis=1)
    if tds_picks.shape[0] >= 50:
        print(tds_picks.shape[0], 'rows')
        for tm in tds_picks.Team.unique():
            display(tds_picks[tds_picks.Team == tm])
    else:
        display(tds_picks)
    tds_picks.insert(0, 'Date', pd.to_datetime(now))
    partition_save_df(tds_picks, f"../tables/2025/gmday_preds_{tgt_stat}.csv")

In [11]:
try: 
    generate_predictions('PTS')
except Exception as e:
    email('PTS', e)
    raise Exception(e)

108 rows


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
2,UTA,Keyonte George,PG,POR,37.062119,35.868445,25.5,33.414925
72,UTA,Lauri Markkanen,PF,POR,37.247974,34.968419,26.5,31.048477
98,UTA,Jusuf Nurkic,C,POR,30.625704,29.032251,11.5,15.568690
99,UTA,Kyle Filipowski,C,POR,21.238546,19.907529,8.5,6.731744
69,UTA,Svi Mykhailiuk,SF,POR,22.548159,18.213153,6.5,8.019393
68,UTA,Brice Sensabaugh,SF,POR,28.752514,25.857176,13.5,12.830184
67,UTA,Ace Bailey,SF,POR,23.459614,23.945918,10.5,10.975637


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
58,CHO,Brandon Miller,SF,OKC,34.110985,33.853749,17.5,25.295433
61,CHO,Kon Knueppel,SF,OKC,31.778009,29.194036,16.5,21.613771
20,CHO,Collin Sexton,SG,OKC,25.168566,23.449282,11.5,16.068108
7,CHO,LaMelo Ball,PG,OKC,28.476482,28.420966,17.5,21.217913
86,CHO,Tidjane Salaun,PF,OKC,17.292379,18.731506,5.5,8.356436
102,CHO,Moussa Diabate,C,OKC,25.731979,26.601913,7.5,8.095120
76,CHO,Miles Bridges,PF,OKC,31.898188,29.365138,16.5,16.661573


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
46,DEN,Julian Strawther,SG,PHI,11.751823,9.084538,14.5,7.052574
45,DEN,Jalen Pickett,SG,PHI,17.721848,15.505867,11.5,5.831013
39,DEN,Bruce Brown,SG,PHI,26.564758,28.340177,11.5,6.175530
62,DEN,Peyton Watson,SF,PHI,31.788095,31.983522,19.5,18.230755


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
19,ATL,Nickeil Alexander-Walker,SG,TOR,36.810467,32.952562,19.5,24.878065
43,ATL,Luke Kennard,SG,TOR,20.870054,13.862179,5.5,9.534451
95,ATL,Onyeka Okongwu,C,TOR,33.884949,33.425439,14.5,17.546986
34,ATL,Dyson Daniels,SG,TOR,36.010021,34.312303,12.5,13.950667
55,ATL,Jalen Johnson,SF,TOR,37.236099,35.115307,23.5,24.701933
65,ATL,Zaccharie Risacher,SF,TOR,26.127592,23.068871,9.5,10.288801
15,ATL,Vit Krejci,PG,TOR,25.181181,24.901602,8.5,8.779696
90,ATL,Kristaps Porzingis,C,TOR,24.386087,25.501392,13.5,13.355462


...

,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
49,TOR,Brandon Ingram,SF,ATL,36.489079,35.344604,23.5,26.826071
83,TOR,Collin Murray-Boyles,PF,ATL,23.903028,21.536039,8.5,6.023293
44,TOR,Ja'Kobe Walter,SG,ATL,21.499193,22.069510,5.5,7.707645
54,TOR,RJ Barrett,SF,ATL,28.626137,29.941568,18.5,16.852867
14,TOR,Jamal Shead,PG,ATL,21.483299,23.681141,6.5,7.846550
74,TOR,Scottie Barnes,PF,ATL,36.413628,35.920688,18.5,19.702631
9,TOR,Immanuel Quickley,PG,ATL,33.699768,33.410746,15.5,16.538509
97,TOR,Sandro Mamukelashvili,C,ATL,26.783232,23.047087,11.5,10.636041


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
105,DET,Paul Reed,C,NYK,19.747416,9.317241,10.5,7.326257
35,DET,Jaden Ivey,SG,NYK,21.046345,19.110493,9.5,7.196956
40,DET,Javonte Green,SG,NYK,24.401085,20.648384,7.5,9.621604
59,DET,Duncan Robinson,SF,NYK,28.684149,28.239597,11.5,9.593971
60,DET,Ausar Thompson,SF,NYK,27.075962,23.903619,12.5,10.632116
5,DET,Cade Cunningham,PG,NYK,36.642853,34.507097,27.5,28.551926
101,DET,Isaiah Stewart,C,NYK,26.947186,23.499428,11.5,11.646321


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
106,HOU,Steven Adams,C,PHO,20.751244,17.485040,7.5,5.307066
82,HOU,Tari Eason,PF,PHO,27.165564,21.672292,14.5,13.767221
11,HOU,Reed Sheppard,PG,PHO,27.993750,25.717914,12.5,13.202938
57,HOU,Amen Thompson,SF,PHO,35.387112,33.921542,18.5,19.065870
51,HOU,Kevin Durant,SF,PHO,36.849957,34.422539,27.5,27.044706
77,HOU,Jabari Smith Jr.,PF,PHO,35.781399,33.634413,15.5,15.888014


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
48,GSW,Jimmy Butler,SF,LAC,34.284580,33.178683,19.5,17.328730
104,GSW,Al Horford,C,LAC,17.267796,14.804859,5.5,7.368828
85,GSW,Quinten Post,PF,LAC,20.430300,18.081041,7.5,8.658999
23,GSW,Brandin Podziemski,SG,LAC,27.455484,28.489386,10.5,11.618134
16,GSW,De'Anthony Melton,PG,LAC,23.145893,20.237901,8.5,9.524648
28,GSW,Moses Moody,SG,LAC,23.452913,22.169621,9.5,8.651485
84,GSW,Draymond Green,PF,LAC,25.782618,25.111067,7.5,8.056100
8,GSW,Stephen Curry,PG,LAC,35.284187,33.970636,27.5,27.548086


../tables/2025/gmday_preds_PTS.csv saved!


In [12]:
try: 
    generate_predictions('PRA')
except Exception as e:
    email('PRA', e)
    raise Exception(e)

82 rows


,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
45,DEN,Jalen Pickett,SG,PHI,17.721848,15.505867,21.5,8.823352
39,DEN,Bruce Brown,SG,PHI,26.564758,28.340177,21.5,13.359382
62,DEN,Peyton Watson,SF,PHI,31.788095,31.983522,27.5,21.241899


,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
95,ATL,Onyeka Okongwu,C,TOR,33.884949,33.425439,25.5,33.352425
90,ATL,Kristaps Porzingis,C,TOR,24.386087,25.501392,19.5,23.636127
34,ATL,Dyson Daniels,SG,TOR,36.010021,34.312303,25.5,27.949669
55,ATL,Jalen Johnson,SF,TOR,37.236099,35.115307,42.5,44.663071
19,ATL,Nickeil Alexander-Walker,SG,TOR,36.810467,32.952562,26.5,27.975611


,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
61,CHO,Kon Knueppel,SF,OKC,31.778009,29.194036,24.5,31.477947
58,CHO,Brandon Miller,SF,OKC,34.110985,33.853749,25.5,31.048805
7,CHO,LaMelo Ball,PG,OKC,28.476482,28.420966,27.5,29.906183
76,CHO,Miles Bridges,PF,OKC,31.898188,29.365138,26.5,27.240915


,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
91,NYK,Karl-Anthony Towns,C,DET,30.658413,29.075297,35.5,42.329121
78,NYK,OG Anunoby,PF,DET,36.960728,35.636049,24.5,26.332893
4,NYK,Jalen Brunson,PG,DET,37.386894,37.777647,39.5,38.371067
53,NYK,Mikal Bridges,SF,DET,38.061649,37.738794,23.5,24.623434
31,NYK,Miles McBride,SG,DET,30.786430,26.204542,17.5,18.001799
107,NYK,Mitchell Robinson,C,DET,20.028292,21.211502,14.5,14.888459


...

,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
104,GSW,Al Horford,C,LAC,17.267796,14.804859,10.5,13.192433
48,GSW,Jimmy Butler,SF,LAC,34.284580,33.178683,29.5,32.038574
23,GSW,Brandin Podziemski,SG,LAC,27.455484,28.489386,17.5,19.445787
84,GSW,Draymond Green,PF,LAC,25.782618,25.111067,18.5,20.186054
28,GSW,Moses Moody,SG,LAC,23.452913,22.169621,13.5,12.034051
85,GSW,Quinten Post,PF,LAC,20.430300,18.081041,12.5,11.755849
8,GSW,Stephen Curry,PG,LAC,35.284187,33.970636,35.5,34.966381


,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
93,BOS,Neemias Queta,C,CHI,26.748314,26.866297,19.5,17.125971
3,BOS,Payton Pritchard,PG,CHI,36.294537,33.326310,26.5,28.027550
47,BOS,Jaylen Brown,SF,CHI,33.844101,31.143381,43.5,41.976841
33,BOS,Anfernee Simons,SG,CHI,26.761539,25.852776,17.5,18.679867
27,BOS,Derrick White,SG,CHI,36.681450,33.894851,29.5,30.532055
100,BOS,Luka Garza,C,CHI,19.659206,19.674841,16.5,16.211889


,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
74,TOR,Scottie Barnes,PF,ATL,36.413628,35.920688,34.5,36.780540
83,TOR,Collin Murray-Boyles,PF,ATL,23.903028,21.536039,16.5,17.861757
97,TOR,Sandro Mamukelashvili,C,ATL,26.783232,23.047087,19.5,20.311523
9,TOR,Immanuel Quickley,PG,ATL,33.699768,33.410746,26.5,25.741850
54,TOR,RJ Barrett,SF,ATL,28.626137,29.941568,27.5,26.791616
49,TOR,Brandon Ingram,SF,ATL,36.489079,35.344604,33.5,33.321499
14,TOR,Jamal Shead,PG,ATL,21.483299,23.681141,13.5,13.548655


,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
11,HOU,Reed Sheppard,PG,PHO,27.993750,25.717914,19.5,18.596502
57,HOU,Amen Thompson,SF,PHO,35.387112,33.921542,32.5,31.735182
77,HOU,Jabari Smith Jr.,PF,PHO,35.781399,33.634413,24.5,25.261282
51,HOU,Kevin Durant,SF,PHO,36.849957,34.422539,38.5,38.056076
82,HOU,Tari Eason,PF,PHO,27.165564,21.672292,22.5,22.916214


../tables/2025/gmday_preds_PRA.csv saved!
